** ClusterAI 2021
** <br>** Ciencia de Datos - Ingenieria Industrial UTN BA
**<br>** Curso I5521
**<br>** Clase 01: Analisis Exploratorio de Datos (EDA) con datos de Google Play Store Apps
**<br>** Elaborado por: Nicolás Aguirre**

## Librerias

In [ ]:
#Importar paquetes de herramientas:

#Datos
import pandas as pd
import numpy as np
#Graficos 
import matplotlib.pyplot as plt
import seaborn as sns
#Otros 
import warnings
warnings.filterwarnings('ignore')

# Dataset

El dataset  que usaremos se encuentra en:
  
https://www.kaggle.com/lava18/google-play-store-apps

Una vez descargado, indicamos la direccion del archivo descargado a la funcion "pd.read_csv()" para importarlo como un objeto Pandas DataFrame. Si el archivo se encuentra en la misma carpeta que la notebook, con indicarle el nombre es suficiente.

Ademas, usaremos la funcion **np.shape()** y y el metodo **.head()** para:

* **Verificar que se haya cargado bien el dataset**: En algunos casos, debido a un error en el formato del archivo ".csv", las columnas y/o registros se cargan incorrectamente. En estos casos "pd.read_csv()" no devuelve error pero lo notaremos cuando usemos la funcion ".head()".


* **Obtener la dimension del dataset**: Cantidad de registros y cantidad de columnas.


* **Tener una base de la cantidad original de registros**: Para que a medida que vayamos aplicando distintos filtros que limpien nuestros datos tengamos una numero de referencia. Si aplicamos un filtro, y de repente perdemos el 90% de los datos, lo mas probable es que en algo nos hayamos equivocado.

In [ ]:
google_df = pd.read_csv('googleplaystore.csv')

In [ ]:
filas = np.shape(google_df)[0] # [0] para la primera dimension
print(f'Filas: {filas}')

columnas = np.shape(google_df)[1] # [1] para la segunda dimension
print(f'Columnas: {columnas}\r\n')

print('Output de ".head(5)": ')
google_df.head(5)

### Columnas
Si queremos saber el nombre de las columas en una variable utilizamos el metodo **.columns()**.

Para guardarlo, simplemente lo asignamos a una variable.

In [ ]:
nombre_columnas = google_df.columns.values
nombre_columnas

# Limpieza de Datos

En esta parte nos vamos a encargar de limpiar:

* **Duplicados** $\rightarrow$  [.drop_duplicates()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html)


* **Simbolos** $\rightarrow$ [str.replace](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.replace.html) and [str.extract](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.extract.html)



* **NaN** $\rightarrow$ [.dropna()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html)


#### Duplicados

In [ ]:
# Vamos a eliminar de la columna 'App' los valores repetidos, conservando unicamente la primera ocurrencia.

# Las columnas para quitar los duplicados se indican en el argumento 'subset'
# mientras que la primera ocurrencia se indica con el argumento 'keep'

google_df.drop_duplicates(subset='App', inplace=True, keep='first')

# La opcion 'inplace' nos evitar tener asignar la salida a una variable.
# Directamente se guarda sobre 'google_df'. Equivale a:

#google_df = google_df.drop_duplicates(subset='App', keep='first')

#### Simbolos

Supongamos que las columnas **Installs**, **Size**, **Price** y **Reviews** tienen informacion en la que estamos interesados.

El problema es que estan en formato texto (o *string*) y hay simbolos y valores que nos impiden manipularlos en formato numerico ( *int* o *float*)

In [ ]:
#Installs
q_installs = len(google_df['Installs'].unique()) # Cantidad de valores unicos en 'Installs'

print(f'En total hay {q_installs} tipos de valores en "Install"\r\n')
print("\n", google_df['Installs'].unique())

Podemos notar 2 simbolos de la cell anterior que debemos eliminar para poder usar el dato como numero y no como cadena de texto, y un valor "Free" en algun registro que esta mal cargado.

Entonces debemos:
1. Conservar los registros que tengan en la colmna 'Install' distinto (!=) a 'Free',
2. Reemplazaar/eliminar los simbolos "+" y ","
3. Cambiar el formato string a int

In [ ]:
#1)
# Eliminacion manual de valores que no deberiamos tener en columas
google_df = google_df[google_df['Installs'] != 'Free']

# Aqui lo que hicimos fue conservar unicamente los registros cuyos valores en la columa "Installs" sean distintos (=!) a 'Free'
filas = np.shape(google_df)[0] # [0] para la primera dimension
print(f'Filas: {filas}')

In [ ]:
#2)
#Reemplaamos los caraccteres '+' y ',' con el comando 'str.replace(a,b)' donde:
# 'a' es el string a reemplazar y 'b' es el string nuevo.  
google_df['Installs'] = google_df['Installs'].str.replace(',','')
google_df['Installs'] = google_df['Installs'].str.replace('+','')

In [ ]:
#3)
google_df = google_df.astype({"Installs": int})

Veamos como quedo nuestra columna 'Install'

In [ ]:
q_installs = len(google_df['Installs'].unique())
print(f'En total hay {q_installs} labels de Install\r\n')
print("\n", google_df['Installs'].unique())

In [ ]:
#Size
q_size = len(google_df['Size'].unique()) # Cantidad de valores unicos en 'Size'
print(f'En total hay {q_size} tipos de valores en "Size"\r\n')
print("\n", google_df['Size'].unique())
filas = np.shape(google_df)[0]
print(f'Filas: {filas}')

En la columna "Size", tenemos problemas de unidades, en algunos casos tenemos el tamaño en Megabytes (M) y en otras en Kilobytes (k), y algunos valores con el texto "Varies with device".

Entonces debemos:
1. Reemplazar los valores 'Varies with device' por NaN's.

2. Separar los numeros que esten en formato texto y guardarlos en formato numerico. Ademas, homogeinizaremos el tamaño a 'M'. Para eso, extraeremos y reemplazaremos los caracteres 'k' y 'M'. 


3. Finalmente, los NaN's correspondiente a los valores "Varies with device", vamos a reemplazarlos por la media de cada categoria, para eliminar la menor cantidad de registros. 

In [ ]:
# 1)
#Dejamos el peso de las app en Mb y convertimos aquellas que esten en Kb.
google_df['Size'].replace('Varies with device', np.nan, inplace = True )

In [ ]:
# 2.1)
#Eliminamos las letras k y M que estan al final de cada valor.

output = google_df.Size.replace(r'[kM]', '', regex=True).astype(float) # Valores enteros sin las letras k/M
print(output,'\r\n')
print(output.value_counts())

In [ ]:
# 2.2)
#Separamos los grupos K y M
output = google_df.Size.str.extract(r'([kM])', expand=False)
print(output,'\r\n')
print(output.value_counts())

In [ ]:
# 2.3)
#Los NaN los reemplazamos por 1 para no perder registros
output = google_df.Size.str.extract(r'([kM])', expand=False).fillna(1)
print(output,'\r\n')
print(output.value_counts())

In [ ]:
# 2.4)
#Homogeneizamos las unidades, k = 10**-3 y M = 1
output = google_df.Size.str.extract(r'([kM])', expand=False).fillna(1)\
.replace(['k','M'], [10**-3, 1]).astype(float)
#print(output,'\r\n')
print(output.value_counts())

In [ ]:
# Juntamos todo [2.1 a 2.4]
google_df.Size = (google_df.Size.replace(r'[kM]', '', regex=True).astype(float) * \
             google_df.Size.str.extract(r'([KM])', expand=False)
            .fillna(1)
            .replace(['k','M'], [10**-3, 1]).astype(float))

# comentario:
# El simbolo ' \ ' permite escribir una misma linea de codigo en distintas lineas de texto

Veamos como quedo ahora la columna 'Size'

In [ ]:
google_df.Size

In [ ]:
# 3)
#Reemplazamos aquellos registros con 'Varies with device' (ahora NaN) con la media del peso segun la categoria
google_df['Size'].fillna(google_df.groupby('Genres')['Size'].transform('mean'), inplace = True)

In [ ]:
# A float
google_df['Size'] = google_df['Size'].astype(float)
google_df['Installs'] = google_df['Installs'].astype(float)

In [ ]:
#quitamos simbolo '$' y pasamos a float.
google_df['Price'] = google_df['Price'].str.replace('$','')
google_df['Price'] = google_df['Price'].astype(float)

In [ ]:
google_df['Reviews'] = google_df['Reviews'].astype(int)

**Links para curiosos**

Regular Expressions:
   - [Documentacion](https://docs.python.org/3/howto/regex.html) 
   - [YouTube](https://www.youtube.com/watch?v=8DvywoWv6fI&list=WL&index=2&t=21317s) 
   - [Ejemplos](https://www.geeksforgeeks.org/pattern-matching-python-regex/)

## NaN

Ahora verificamos que las columnas no tengan NaN.

En caso de haberlos, tendremos que decidir:

- si son suficientes como para eliminar TODA la columa y perder esa informacion, o

        
- decidir eliminar unicamente los registros.

In [ ]:
# Columnas que tienen al menos un nan
col_NaN = google_df.isnull().any()
print(col_NaN,'\r\n')

In [ ]:
# Cantidad de valores nulos ordenados descendentemente
total = google_df.isnull().sum().sort_values(ascending=False)
total

In [ ]:
# Cantidad de valores nulos ordenados descendentemente
total = google_df.isnull().sum().sort_values(ascending=False)
# Porcetaje de lo que representa para cada columna
percent = (google_df.isnull().sum()/google_df.isnull().count()).sort_values(ascending=False)
# Mostramos los 2 resultados en conjunto.
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(6)

In [ ]:
# Limpiamos registros '.dropna'
google_df.dropna(how ='any', inplace = True)

In [ ]:
# Verificamos como quedo el dataset
print(f'Nos quedamos con un dataframe de {google_df.shape[0]} filas x {google_df.shape[1]} columnas')

### Ahora que terminamos con la limpieza general del dataset, pasemos a analizar:
    
    1. Rating
    
    2. Categorias
    
    3. Categorias vs Rating
    
    4. Reviews
    
    5. Precio

# 1) Rating

A continuacion veamos la distribucion estimada del 'Rating' y su histograma

In [ ]:
plt.figure(figsize=(10,3))
sns.kdeplot(google_df['Rating'], color= "Blue", shade = True)
plt.xlabel("Rating",size = 20)
plt.ylabel("Frecuency",size = 20)
plt.title('Distribution of Rating',size = 20)
#plt.imsave('KDE',format='png') # Guardar la imagen
plt.show()

plt.figure(figsize=(10,3))
sns.countplot(google_df.Rating, color="Blue")
plt.xlabel("Rating",size = 20)
plt.ylabel("Qty",size = 20)
plt.title('Histogram of Rating',size = 20)
plt.xticks([]) # por si deseamos eliminar los intervalos en el eje-x
#plt.imsave('Histogram',format='png') # Guardar la imagen
plt.show()

# 2) Categorias

A modo de ejemplo, podriamos querer saber como es el comportamiento del Rating por Categoria:
    
* Vamos a visualizar el top 10 de categorias con mayor cantidad de apps.

* Boxplot de Categoria vs Rating

In [ ]:
# Gardamos en una variable la cantidad de categorias
q_categorias = len(google_df['Category'].unique())
print(f'En total hay {q_categorias} categorias\r\n')
# Mostramos en la cell los tipos de categorias
print("\n", google_df['Category'].unique())

In [ ]:
# top
top_n = 10
# Guardamos los indices de los top_n categorias
idx_top = google_df['Category'].value_counts(ascending=False).index[0:top_n]

plt.figure(figsize=(15,3))
# Indicamos la columna 'Category', de los datos 'google_df', en el orden 'idx_top'
g = sns.countplot(x="Category", data=google_df,
                  order=idx_top,
                  palette = "muted")
g.set_xticklabels(g.get_xticklabels(), rotation=90, ha="right")
plt.title('Cantidad de App por Categorias', size = 20)
plt.xlabel("Categoria", size = 20)
plt.ylabel("Cantidades", size = 20)
plt.show()

# 3) Categoria vs Rating

In [ ]:
# Todas las categorias
g = sns.catplot(x="Category",y="Rating",
                data=google_df, 
                kind="box",
                order=idx_top,
                palette = "muted",
                height = 5 ,aspect=3)

g.despine(left=True) # Para quitar linea del Y del plot
g.set_xticklabels(rotation=90)
plt.xlabel("Categoria",size = 20)
plt.ylabel("Rating",size = 20)
plt.title('Boxplot de Rating VS Categorias',size = 20)
plt.show()

## Mediana, Q1-Q3, whiskers y  outliers 
En muchas ocaciones, luego de ver los plots vamos a querer guardar en variables valores como la media, los valores atipicos y los "whiskers" para cada categoria.

A modo de ejemplo, veamos como obtenerlos para una unica categoria.

In [ ]:
cat_select = 'LIFESTYLE'
df_pivot = google_df[google_df['Category']==cat_select]
plt.figure()
bxplot = plt.boxplot(df_pivot['Rating'])
#bxplot = plt.boxplot(df_pivot['Rating'],whis=[15, 82])
plt.show()

In [ ]:
# Mediana
medians = bxplot["medians"][0].get_ydata()

# Marcas de Boxplot
low_limits = bxplot["whiskers"][0].get_ydata()
up_limits = bxplot["whiskers"][1].get_ydata()

# Valores Q1 - Q3
Q1 =  low_limits[0]
Q3 =  up_limits[0]

# whiskers: Valores extremos de las lineas que salen del intervalo [Q1-Q3]
low_whiskers =  low_limits[1]
up_whiskers = up_limits[1]

# Outliers = fliers: Valores mas alla de los whiskers 
outliers = bxplot["fliers"][0].get_ydata()

In [ ]:
print(f'Valor Mediana:\r\n {medians[0]}\r\n')
print(f'[Q1 - Q3] : [{Q1} - {Q3}]\r\n')
print(f'[Inferior  - Superior]: [{low_whiskers} - {up_whiskers}] \r\n')
print(f'Valores outliers:\r\n{outliers}\r\n')

# 4) Reviews

In [ ]:
# En este primer plot vamos a ver una estimacion de la distribucion de de los reviews de la cantidad de Reviews
plt.figure(figsize=(10,3))
g = sns.kdeplot(google_df.Reviews, color="Green", shade = True)
plt.xlabel("Reviews",size = 20)
plt.ylabel("Frecuency",size = 20)
plt.title('Distribution of Reveiw',size = 20)
plt.show()

# En este segundo plot vamos a ver el histograma de la cantidad de reviews.
# El parametro bins define la cantidad de sub-intervalos en los que vamos a dividir el eje-x
plt.figure(figsize=(10,3))
plt.hist(google_df['Reviews'], bins=100,color='g' ,alpha=0.5)
plt.xlabel("Reviews",size = 20)
plt.ylabel("Cantidades",size = 20)
plt.show()

Este ultimo grafico nos da alguna informacion?

Muchas veces los graficos parecen que no nos muestran nada.

En los casos donde tenemos muchas informacion concentrada, una buena practica es hacer un cambio de escala.

In [ ]:
plt.figure(figsize=(10,3))
plt.hist(np.log(1+google_df.Reviews),bins=100, color='g' ,alpha=0.5)
plt.xlabel("Log(Reviews)",size = 20)
plt.ylabel("Cantidades",size = 20)
plt.show()

Ahora podemos ver mejor como se distribuyen las cantidades de reviews ...

Alla en el fondo, donde antes no veiamos nada, ahora podemos ver que hay un par de apps con muchisimos reviews...
veamos cuales son ...

In [ ]:
google_df[google_df.Reviews > 5000000].head()

**Habra alguna relacion entre los "Reviews" y alguna otra variable?**

In [ ]:
corrmat = google_df.corr()

f, ax = plt.subplots(figsize=(9, 7))
ax = sns.heatmap(corrmat,
               annot=True,
               cmap=sns.diverging_palette(240, 10, as_cmap=True))

**Tiene sentido?**

# 5) Precio

Tenemos a nuestra disposicion tambien los precios de las Apps, asi que vamos a usarlos!

* Estadistica descriptiva que nos da [**.describe()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html) y si algo nos llama la atencion, explorarlo y si hace falta corregirlo.


* Vamos a armar a criterio nuestro categorias de precio de las apps.


* Crucemos informacion entre las categorias de precio Precio y Rating, y saquemos conclusiones.

In [ ]:
google_df['Price'].describe()

#### Hay una app que cuesta U$D 400 ! !

In [ ]:
google_df[google_df['Price'] == 400]

### Ahora armemos a nuestro gusto brands segun los precios y veamos como se distribuyen

In [ ]:
#Primero defininmos los limites de cada categoria y creamos la columna 'PriceBand'

google_df.loc[ google_df['Price'] == 0, 'PriceBand'] = '0 Free'
google_df.loc[(google_df['Price'] > 0) & (google_df['Price'] <= 0.99), 'PriceBand'] = '1 Muy Barato'
google_df.loc[(google_df['Price'] > 0.99) & (google_df['Price'] <= 2.99), 'PriceBand']   = '2 Barato'
google_df.loc[(google_df['Price'] > 2.99) & (google_df['Price'] <= 4.99), 'PriceBand']   = '3 Normal'
google_df.loc[(google_df['Price'] > 4.99) & (google_df['Price'] <= 14.99), 'PriceBand']   = '4 Caro'
google_df.loc[(google_df['Price'] > 14.99) & (google_df['Price'] <= 29.99), 'PriceBand']   = '5 Muy Caro'
google_df.loc[(google_df['Price'] > 29.99), 'PriceBand']  = '6 #VamoACalmarnos'

Nota: **df.loc()** filtra por labels (y por eso se pueden usar 'nombres' de columna/filas) mientras que **df.iloc()** es como usar arrays (e.g.,  [0:2,3:-1] )

Se recomienda leer y entender la diferencia entre los metodos ya que probablemente los usen continuamente.
* [**df.loc()**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html)
* [**df.iloc()**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iloc.html)

In [ ]:
#Veamos como es el mean value para las bandas que definimos
google_df[['PriceBand', 'Rating']].groupby(['PriceBand'], as_index=False).mean()

In [ ]:
# Ahora en vez de utilizar 'box'(boxplot)
# ingresaremos 'boxen' al argumento 'kind' de la funcion 'catplot'
g = sns.catplot(x="PriceBand", y="Rating",
                data=google_df, 
                kind="boxen", # box , violin
                height = 5,aspect=3 ,palette = "muted")
g.despine(left=True)
g.set_xticklabels(rotation=90)
g = g.set_ylabels("Rating")
plt.title('Boxen plot Rating VS PriceBand',size = 20)
plt.show()

Para bases de datos muy grandes, muchas veces los limites de confiabilidad del boxplot por defecto consideran erroneamente "outliers" a muestras con muy baja probabilidad, que si deberian considerarse como parte del espacio de muestra.

Ademas, el "boxplot" no deja visualizar como se distribuyen de muestras en los extremos.

Si creemos que alguno de estos factores, entre otros, nos puede estar sucediendo y nos esconde informacion que creemos relevante mostrar, lo mejor es probar con algun otro tipo de ploteo ([boxen](https://vita.had.co.nz/papers/letter-value-plot.pdf) o  "violin").

# Propuesta:
    
    a. Apps Pagas vs Apps Free
    
    b. Content Rating (Everyone, Teen, +18, etc)
         Hint: google_df['Genres'] = google_df['Genres'].str.split(';').str[0]
         
    c. Genres vs Rating
    
    d. Genres (Estadistica descriptiva w.r.t , i.e, "Rating")
    
    e. Mismo analisis, pero en vez de reemplazar Varies with device por la media de w.r.t. categoria, 
    eliminando los registros y ver si el supuesto que hicimos impacta en los resultados. 